In [30]:
import numpy as np
import progressbar

In [31]:
def split_to_segments(samples, num_segments):
    """
    Splits an incoming data to a set of segments
    :param samples: incoming samples
    :param num_segments: number of segments to be divided into
    :returns an object array for split up segments    
    """
    num_samples_per_segment = samples.size//num_segments
    new_samples = np.split(samples[:num_segments*num_samples_per_segment], num_segments)
    new_samples[-1] = np.vstack([new_samples[-1], samples[num_segments*num_samples_per_segment:]])
    return new_samples

In [32]:
def test_samples_after_pooling(samples, num_segments):
    """
    Tests samples     
    :param samples: incoming samples
    :param num_segments: number of segments to be divided into for testing
    :returns total number fo tests needed    
    """
    if np.all(samples==False) or samples.size == 1:
        return 1
    else:
        segments = split_to_segments(samples, num_segments)
        return np.sum([test_samples_after_pooling(segment, num_segments)
                       for segment in segments]) + 1

In [42]:
def generate_samples(population_size, infection_probability, sort_data):
    """
    generates samples based on infection probablity for each sample. also sorts the data if sort_data is true
    """
    I = np.argsort(infection_probability)
    samples = np.asarray([np.random.choice([True, False], 1, p=[infection_probability[i], 
                                                                1-infection_probability[i]])
                           for i in range(population_size)])
    if sort_data:
        return samples[I]
    else:
        return samples

In [44]:
def monte_carlo_simulation(population_size, num_iterations, infection_probability, num_segments, 
                           pool_sample_probabilities=False):
    """
    Carries out a monte carlo simulation
    """
    if pool_sample_probabilities is True:
        sample_probabilities = np.random.random(population_size)*infection_probability*2
    else:
        sample_probabilities = np.ones(population_size)*infection_probability
    average_num_tests = 0
    for i in range(num_iterations):
        samples = generate_samples(population_size, sample_probabilities, pool_sample_probabilities)
        num_tests = test_samples_after_pooling(samples, num_segments)
        average_num_tests += num_tests/population_size/num_iterations
    return average_num_tests

In [ ]:
total_population = 10000
num_segments = 2
infection_probability = 0.03
print('If we pool by sorting: ' + 
      str(monte_carlo_simulation(total_population, 100, infection_probability, num_segments, True)))
print('If we dont pool by sorting: ' + 
      str(monte_carlo_simulation(total_population, 100, infection_probability, num_segments, False)))